## Finding good outcomes with oracles

In [1]:
from math import sqrt
n = 3
# equally probable outcomes
state = [1/sqrt(2**n) for _ in range(2**n)]

In [2]:
# classical implementation of a phase oracle
# outcome 3 is the only good outcome
predicate = lambda k: True if k == 3 else False

In [3]:
# apply phase oracle to state
def oracle(state, predicate):
  for item in range(len(state)):
    if predicate(item):
      state[item] *= -1

oracle(state, predicate)

In [4]:
# resulting states
state

[0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 -0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373]

## Computing similarity with inner products

In [5]:
# simple inner / dot prod example
quantities = [4, 2, 2, 3]
prices = [1.2, 1.5, 2, 0.7]

# total price / inner product between these two vectors
print(sum([quantities[k] * prices[k] for k in range(len(quantities))]))

13.9


In [6]:
# The inner product of two complex vectors is defined as the sum of the products of
# each element in the first vector and the conjugate of the corresponding element in
# the second vector.

# computes inner prod of two state vectors
def inner(v1, v2):
  assert(len(v1) == len(v2))
  return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

In [7]:
# What is the inner product of i, i and –i, i?

# a = (i,i)
# b = (-i, i)

# conjugate of b = (i, -i)

# therefore inner prod = (i)(i) + (-i)(i) = 0
# this means the two vectors are orthogonal in the complex vector space

## The inversion operator

In [8]:
# Classical implementation of the inversion / reflection operator
def inversion(original, current):
  proj = inner(original, current)
  for k in range(len(current)):
    current[k] = 2*proj*original[k] - current[k]

In [9]:
# applying an oracle that tags the outcome 3 to a state where the amplitudes are in
# equal superposition
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]
s = state.copy()

oracle(state, predicate)

state

[0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 -0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373]

In [10]:
# apply the inversion operator that reflects the current state into the original
# state (the state before the oracle was applied)
inversion(s, state)

state

[0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675,
 0.8838834764831842,
 0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675]

In [11]:
# another example, with a random n = 3 qubit state and good outcome 5
from math import sqrt, cos, sin, pi

import random


def generate_state(n, seed=555):
    # Choose a seed
    random.seed(seed)
    # Generate 4 random probabilities that add up to 1
    probs = [random.random() for _ in range(2 ** n)]
    total = sum(probs)
    probs = [p / total for p in probs]
    # Generate 4 random angles in radians
    angles = [random.uniform(0, 2 * pi) for _ in range(2 ** n)]
    # Build the quantum state array
    return [sqrt(p) * (cos(a) + 1j * sin(a)) for (p, a) in zip(probs, angles)]

n = 3
state = generate_state(n)
s = state.copy()
predicate = lambda k: True if k == 5 else False
oracle(state, predicate)

state

[(-0.1850274003173231-0.13415359909602925j),
 (0.20703422223707688+0.025419796255241673j),
 (0.27264973801555115+0.2779868749583435j),
 (0.48666008295331065+0.10795909579034865j),
 (-0.2027996224373796+0.01768188885770351j),
 (-0.32193555635783244-0.08714885571466746j),
 (-0.1711590838553813-0.34470771234522085j),
 (0.318969284654943+0.3187672312604926j)]

In [12]:
# perform inversion
inversion(s, state)

state

[(-0.10269951368444127-0.07446199515612928j),
 (0.11491440674905093+0.014109265486591211j),
 (0.15133431833536848+0.15429669778593635j),
 (0.2701208240683233+0.05992272829032866j),
 (-0.11256399086831487+0.009814337679685148j),
 (0.8225615392091293+0.2226697097641888j),
 (-0.09500190049947127-0.19133011846038478j),
 (0.1770439965008243+0.17693184670402257j)]

## Visualizing inversion by the mean

In [13]:
# create an n = 3 qubit state in equal superposition and apply an
# oracle for good outcome 3
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]
s = state.copy()
predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

state

[0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 -0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373]

In [14]:
# check that the mean of the amplitudes is equal to the quantity defined previously
# for bad outcomes k
def is_close_float(a, b, rtol=1e-5, atol=1e-8):
    return abs(a - b) < atol + rtol * abs(b)


def is_close(a, b):
    if isinstance(a, float):
        a = complex(a, 0)

    if isinstance(b, float):
        b = complex(b, 0)

    return is_close_float(a.real, b.real) and is_close_float(a.imag, b.imag)

amplitude_mean = sum(state)/2**n

proj = inner(s, state)

for k in range(len(state)):
  if k != 3:
    assert is_close(proj*state[k], amplitude_mean)

In [15]:
# simulate the inversion by the mean
for k in range(len(state)):
  state[k] = 2*amplitude_mean-state[k]

state

[0.17677669529663687,
 0.17677669529663687,
 0.17677669529663687,
 0.8838834764831843,
 0.17677669529663687,
 0.17677669529663687,
 0.17677669529663687,
 0.17677669529663687]

## Putting it together: The Grover iterate

In [16]:
# Classical implementation of the Grover iterate
from math import cos

def classical_grover(state, predicate, iterations):
  s = state.copy()
  items = [k for k in range(len(state)) if predicate(k)]
  p = sum([abs(s[k])**2 for k in items]) # Uses the probability of
                                         # measuring a good outcome
                                         # to define an angle theta
  theta = asin(sqrt(p))
  assert is_close(inner(s, state), 1)

  for it in range(1, iterations + 1):
    oracle(state, predicate)
    inversion(s, state)

    assert is_close(inner(s, state), cos(2 * it * theta)) # The inner product of the state after operator
                                                          # A is applied and the state after j applications
                                                          # of the Grover iterate is cos(2jθ ).


    p = sum([abs(state[k])**2 for k in items]) # New prob of measuring good outcome

    assert is_close(p, sin((2 * it + 1)*theta)**2) # Check that prob of good outcome is
                                                   # sin^2((2j+1)θ )

In [17]:
# n = num qubits
# l = num good outcomes
# j = num iterations

from math import sin, asin
def target_amplitude_uniform(n, l, j):
  theta = asin(sqrt(l/2**n))
  return sin((2*j+1)*theta)/sqrt(l)

In [18]:
# apply one iteration of the Grover operator to our example state, where n = 3 and
# outcome 3 is the good outcome

n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 1)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 1))

In [19]:
state # check state

[0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675,
 0.8838834764831842,
 0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675,
 0.17677669529663675]

In [20]:
# apply another iteration
n = 3
items = [3]
predicate = lambda i: True if i in items else False
state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 2)
assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 2))

state

[-0.08838834764831847,
 -0.08838834764831847,
 -0.08838834764831847,
 0.9722718241315025,
 -0.08838834764831847,
 -0.08838834764831847,
 -0.08838834764831847,
 -0.08838834764831847]

In [21]:
# three iterations
n = 3
items = [3]
predicate = lambda i: True if i in items else False
state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 3)
assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 3))

state

[-0.3093592167691144,
 -0.3093592167691144,
 -0.3093592167691144,
 0.5745242597140696,
 -0.3093592167691144,
 -0.3093592167691144,
 -0.3093592167691144,
 -0.3093592167691144]

In [22]:
# find optimal number of iterations
from math import floor, pi

num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

In [23]:
num_iterations

2

In [24]:
# another example, this time using a random two-qubit state and good outcome 1
n = 2
items = [1]
predicate = lambda i: True if i in items else False
num_iterations = 3

state = generate_state(n)

for it in range(1, num_iterations + 1):
  s = state.copy()
  classical_grover(s, predicate, iterations = it)

s

[(-0.09870891446551384-0.1421451767685894j),
 (-0.7316894823662417+0.45306740264753737j),
 (0.28137852886624753+0.08808011829251601j),
 (-0.0065053082794929475+0.37740883163769834j)]

In [26]:
# create a random state with n = 4 qubits, and apply the classical magnitude
# amplification procedure for good outcomes 3 and 10.
n = 4
items = [3, 10]
predicate = lambda i: True if i in items else False
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

state = generate_state(n)

for it in range(1, num_iterations + 1):
  s = state.copy()
  classical_grover(s, predicate, iterations = it)

s

[(0.05387900751556492-0.10392592124190086j),
 (0.08808771967712664+0.060461190641004955j),
 (-0.13055299625138053+0.15077514128217978j),
 (-0.6546112269426351+0.39195166686703486j),
 (0.004756688423812804-0.10416121315733856j),
 (0.07574160536504546-0.15312473985457967j),
 (-0.06647241569192103+0.1855842434110626j),
 (0.15346770310371383+0.1726241748101643j),
 (-0.007682739146303109+0.2060474411545934j),
 (0.0137203364905286+0.0344942604472736j),
 (0.26181603815736587+0.10741708562844107j),
 (0.028367683596480536+0.040715791589976194j),
 (-0.030458513861033043-0.18310997148204494j),
 (0.012901751171884607-0.053062921041685265j),
 (0.09178174883590404-0.19883256016621345j),
 (0.015068600060063204+0.09289169833847918j)]

In [27]:
num_iterations

2

## A classical but quantum-friendly implementation of the inversion operator

In [28]:
# helper functions
import numpy as np


def random_transformation(n):
    import scipy.stats
    U = scipy.stats.unitary_group.rvs(2**n)

    def f_direct(state):
        assert(len(state) == 2**n)
        s = U @ state
        for k in range(len(s)):
            state[k] = s[k]

    def f_inverse(state):
        assert(len(state) == 2**n)
        s = np.conj(U.transpose()) @ state
        for k in range(len(s)):
            state[k] = s[k]

    return f_direct, f_inverse


def inner(v1, v2):
    assert(len(v1) == len(v2))
    return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

In [55]:
# create a random transformation (and its inverse) for n = 3 qubits
n = 3

f = random_transformation(n)

A = f[0]
A_inverse = f[1] # first val is random transformation, second is its inverse

In [56]:
# look at the state prepared by this random operator. We do this by initializing a
# state and then applying the random transformation
def init_state(n):
    state = [0 for _ in range(2 ** n)]
    state[0] = 1
    return state

state = init_state(n)

A(state)

In [57]:
state

[np.complex128(-0.1509189321271005+0.06463067520657557j),
 np.complex128(0.23915461200874952-0.017876117524369097j),
 np.complex128(-0.20494310876226815+0.08118761170589206j),
 np.complex128(-0.2796172578591335-0.06815105124603998j),
 np.complex128(0.09443810021640014+0.11725881067296018j),
 np.complex128(-0.2115152679502047+0.35604318121377093j),
 np.complex128(0.34350638426989133+0.5688783994862742j),
 np.complex128(-0.2300502225029217+0.30885655520845634j)]

In [58]:
# Function to perform the inversion operation
from math import log2

def inversion_0_transformation(f, state):
  n = int(log2(len(state)))

  transform = f[0] # operation A
  inverse_transform = f[1] # operation A^-1

  inverse_transform(state) # applies inverse operation

  assert is_close(state[0].imag, 0)

  for k in range(1, len(state)):
    state[k] = -state[k] # Applies Mo

  transform(state) # applies A

In [59]:
# use the same operator A we used to create the state and apply an
# oracle for the good outcome 3
predicate = lambda k: True if k == 3 else False

oracle(state, predicate)

# apply inverse transform to state
inversion_0_transformation(f, state)

state

[np.complex128(-0.10091624414302036+0.04321714251716362j),
 np.complex128(0.15991754562031146-0.011953375331991298j),
 np.complex128(-0.13704104917641916+0.054288409868951656j),
 np.complex128(-0.7462085624001618-0.18187324475511124j),
 np.complex128(0.06314872656145631+0.07840844484525566j),
 np.complex128(-0.1414357106798415+0.23807841795863413j),
 np.complex128(0.22969533146757326+0.3803967510312022j),
 np.complex128(-0.1538296361632694+0.20652573597816948j)]

## Magnitude amplification: Quantum circuit implementation

In [86]:
## helper functions
x = [[0, 1], [1, 0]]

z = [[1, 0], [0, -1]]


def phase(theta):
    return [[1, 0], [0, complex(cos(theta), sin(theta))]]


h = [[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]]


def rz(theta):
    return [[complex(cos(theta / 2), -sin(theta / 2)), 0], [0, complex(cos(theta / 2), sin(theta / 2))]]


y = [[0, complex(0, -1)], [complex(0, 1), 0]]


def rx(theta):
    return [[cos(theta/2), complex(0, -sin(theta/2))], [complex(0, -sin(theta/2)), cos(theta/2)]]


def ry(theta):
    return [[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]]

def is_power_of_two(m):
    return ceil(log2(m)) == floor(log2(m))


def prepare_state(*a):
    state = [a[k] for k in range(len(a))]
    assert(is_power_of_two(len(state)))
    assert(is_close(sum([abs(state[k])**2 for k in range(len(state))]), 1.0))
    return state


def init_state(n):
    state = [0 for _ in range(2 ** n)]
    state[0] = 1
    return state


def is_bit_set(m, k):
    return m & (1 << k) != 0


def pair_generator_check_digit(n, t):
    distance = int(2 ** t)

    for k0 in range(2 ** n):
        if not is_bit_set(k0, t):
            k1 = k0 + distance
            yield k0, k1


def pair_generator_concatenate(n, t):
    distance = int(2 ** t)
    suffix_count = int(2 ** t)
    prefix_count = int(2 ** (n - t - 1))

    for p in range(prefix_count):
        for s in range(suffix_count):
            k0 = p * suffix_count*2 + s
            k1 = k0 + distance
            yield k0, k1


def pair_generator_pattern(n, t):
    distance = int(2 ** t)

    for j in range(2 ** (n-t-1)):
        for k0 in range(2*j*distance, (2*j+1)*distance):
            k1 = k0 + distance
            yield k0, k1


pair_generator = pair_generator_concatenate


def process_pair(state, gate, k0, k1):
    x = state[k0]
    y = state[k1]
    # new amplitudes
    state[k0] = x * gate[0][0] + y * gate[0][1]
    state[k1] = x * gate[1][0] + y * gate[1][1]


def transform(state, t, gate):
    n = int(log2(len(state)))
    for (k0, k1) in pair_generator(n, t):
        process_pair(state, gate, k0, k1)


def c_transform(state, c, t, gate):
    n = int(log2(len(state)))
    for (k0, k1) in filter(lambda p: is_bit_set(p[0], c), pair_generator(n, t)):
        process_pair(state, gate, k0, k1)


def mc_transform(state, cs, t, gate):
    assert t not in cs
    n = int(log2(len(state)))
    for (k0, k1) in filter(lambda p: all([is_bit_set(p[0], c) for c in cs]), pair_generator(n, t)):
        process_pair(state, gate, k0, k1)


def measure(state, shots):
    samples = choices(range(len(state)), [abs(state[k])**2 for k in range(len(state))], k=shots)
    counts = {}
    for (k, v) in Counter(samples).items():
        counts[k] = v
    return counts

class QuantumRegister:
    def __init__(self, size, shift=0):
        self.size = size
        self.shift = shift

    def __getitem__(self, key):
        if isinstance(key, slice):
            return [self[ii] for ii in range(*key.indices(len(self)))]
        elif isinstance(key, int):
            if key < 0:
                key += len(self)
            assert(0 <= key < self.size)
            return self.shift + key

    def __len__(self):
        return self.size

    def __iter__(self):
        return list([self.shift + i for i in range(self.size)])

    def __reversed__(self):
        return list([self.shift + i for i in range(self.size)[::-1]])


class QuantumTransformation:
    def __init__(self, gate, target, controls=[], name=None, arg=None):
        self.gate = gate
        self.target = target
        self.controls = controls
        self.name = name
        self.arg = arg

    def __str__(self):
        return rf'{self.name} {round(self.arg, 2) if self.arg is not None else ""} {self.controls} {self.target}'

    def __copy__(self):
        return QuantumTransformation(self.gate, self.target, self.controls, self.name, self.arg)


class QuantumCircuit:
    def __init__(self, *args):
        bits = 0
        regs = []
        for register in args:
            register.shift = bits
            bits += register.size
            regs.append(register.size)

        self.state = init_state(bits)
        self.transformations = []
        self.regs = regs
        self.reports = {}

    def x(self, t):
        self.transformations.append(QuantumTransformation(x, t, [], 'x'))

    def y(self, t):
        self.transformations.append(QuantumTransformation(y, t, [], 'y'))

    def z(self, t):
        self.transformations.append(QuantumTransformation(z, t, [], 'z'))

    def h(self, t):
        self.transformations.append(QuantumTransformation(h, t, [], 'h'))

    def p(self, theta, t):
        self.transformations.append(QuantumTransformation(phase(theta), t, [], 'p', theta))

    def rx(self, theta, t):
        self.transformations.append(QuantumTransformation(rx(theta), t, [], 'rx', theta))

    def ry(self, theta, t):
        self.transformations.append(QuantumTransformation(ry(theta), t, [], 'ry', theta))

    def rz(self, theta, t):
        self.transformations.append(QuantumTransformation(rz(theta), t, [], 'rz', theta))

    def cx(self, c, t):
        self.transformations.append(QuantumTransformation(x, t, [c], 'x'))

    def cy(self, c, t):
        self.transformations.append(QuantumTransformation(y, t, [c], 'y'))

    def cz(self, c, t):
        self.transformations.append(QuantumTransformation(z, t, [c], 'z'))

    def cp(self, theta, c, t):
        self.transformations.append(QuantumTransformation(phase(theta), t, [c], 'p', theta))

    def cry(self, theta, c, t):
        self.transformations.append(QuantumTransformation(ry(theta), t, [c], 'ry', theta))

    def mcx(self, cs, t):
        self.transformations.append(QuantumTransformation(x, t, cs, 'x'))

    def mcp(self, theta, cs, t):
        self.transformations.append(QuantumTransformation(phase(theta), t, cs, 'p', theta))

    def measure(self, shots=0):
        state = self.run()
        samples = measure(state, shots)
        return {'state vector': state, 'counts': samples}

    def run(self):
        for tr in self.transformations:
            cs = tr.controls
            if len(cs) == 0:
                transform(self.state, tr.target, tr.gate)
            elif len(cs) == 1:
                c_transform(self.state, cs[0], tr.target, tr.gate)
            else:
                mc_transform(self.state, cs, tr.target, tr.gate)
        self.transformations = []
        return self.state


    def initialize(self, state):
        self.state = state

    def append(self, circuit, reg):
        assert(reg.size == sum(circuit.regs))
        for tr in circuit.transformations:
            self.transformations.append(QuantumTransformation(tr.gate, reg.shift + tr.target, tr.controls, tr.name, tr.arg))

    def c_append(self, circuit, c, reg):
        assert(c not in range(reg.shift, reg.shift + reg.size))
        for tr in circuit.transformations:
            self.transformations.append(QuantumTransformation(tr.gate, reg.shift + tr.target,
                                                              [c] + [reg.shift + t for t in tr.controls],
                                                              tr.name, tr.arg))


    def inverse(self):
        qs = [QuantumRegister(size, 'q' if len(self.regs) == 1 else None) for size in self.regs]
        qc = QuantumCircuit(*qs)

        for tr in self.transformations[::-1]:
            prefix = ''
            if len(tr.controls) == 1:
                prefix = 'c'
            elif len(tr.controls) > 1:
                prefix = 'mc'

            m = getattr(qc, prefix + tr.name)
            cs = tr.controls

            t = tr.target
            reg = 0
            while t >= self.regs[reg]:
                t = t - self.regs[reg]
                reg = reg + 1

            if len(cs) == 0:
                if tr.arg:
                    m(-tr.arg, qs[reg][t])
                else:
                    m(qs[reg][t])
            elif len(cs) == 1:
                if tr.arg:
                    m(-tr.arg, cs[0], qs[reg][t])
                else:
                    m(cs[0], qs[reg][t])
            # multi-control
            else:
                if tr.arg is not None:
                    m(-tr.arg, cs, qs[reg][t])
                else:
                    m(cs, qs[reg][t])

        return qc

    def report(self, name=None):
        start_state = init_state(sum(self.regs))
        tr_count = 0
        for report in self.reports.values():
            if report[3] > tr_count:
                tr_count = report[3]
                start_state = report[2]

        qc = QuantumCircuit()
        qc.regs = self.regs.copy()
        qc.initialize(start_state.copy())
        qc.transformations = self.transformations[tr_count:].copy()
        end_state = qc.run()

        if name is None:
            name = len(self.reports)
        report = (start_state, self.transformations[tr_count:len(self.transformations)].copy(), end_state,
                  len(self.transformations))
        self.reports[name] = report
        return report

In [77]:
# Function to create a phase oracle circuit
def is_bit_not_set(m, k):
  return not (m & (1 << k))

def phase_oracle_match(n, items):
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)
  for m in items:
    for i in range(n):
      if is_bit_not_set(m, i):
        qc.x(q[i])
    qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])
    for i in range(n):
      if is_bit_not_set(m, i):
        qc.x(q[i])
  return qc

In [78]:
# Function to create a circuit for n qubits that multiplies outcome 0 by –1
def inversion_0_circuit(n):
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)

  for i in range(n):
    qc.x(q[i])

  qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

  for i in range(n):
    qc.x(q[i])

  return qc

In [79]:
# Function to create the inversion circuit
def inversion_circuit(A):
  n = sum(A.regs)
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)
  qc.append(A.inverse(), q)
  qc.append(inversion_0_circuit(n), q)
  qc.append(A, q)
  return qc

## Grover iterate

In [80]:
# Function to create the Grover iterate circuit
def grover_iterate_circuit(A, O):
  n = sum(O.regs)
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)
  qc.append(O, q)
  qc.append(inversion_circuit(A), q)
  return qc

## Putting it all together: Grover’s algorithm

In [81]:
# Function to create the magnitude amplification circuit
def grover_circuit(A, O, iterations):
  n = sum(A.regs)
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)

  qc.append(A, q)
  for i in range(1, iterations + 1):
    qc.append(grover_iterate_circuit(A, O), q)
    qc.report(f'iteration_{i}')

  return qc

Suppose we have a list of N = 8 items, represented as the outcomes of a quantum computation
with n = 3 qubits, and an oracle tagging three good outcomes. We can use
the uniform function from chapter 4 to create a circuit A that prepares a state with
equal magnitudes:

In [82]:
def uniform(n):
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)
  for i in range(len(q)):
    qc.h(q[i])
  return qc

In [89]:
# With uniform as our operator A, n = 3 qubits, and good outcomes 1, 3, and 7, we can
# define the following circuit

n = 3 # num qubits
items = [1, 3, 7] # good outcomes

num_iterations = int(floor(pi/4*sqrt(2**n/len(items)))) # opt number of iterations

qc = grover_circuit(uniform(n), phase_oracle_match(n, items), num_iterations)

In [91]:
# check the amplitudes at each step using the reports generated
for i in range(1, num_iterations + 1):
  for m in items:
    assert is_close(
      qc.reports[f'iteration_{i}'][2][m],
      (-1)**i * target_amplitude_uniform(n, len(items), i)
    )

In [93]:
# print out amplitudes
for name, (start, ops, end_state, _) in qc.reports.items():
    print(f"=== Report: {name} ===")
    print(" amplitudes:", end_state)
    print()

=== Report: iteration_1 ===
 amplitudes: [(0.17677669529663678-2.1648901405887307e-17j), (-0.5303300858899103+2.1648901405887317e-17j), (0.17677669529663678-2.1648901405887307e-17j), (-0.5303300858899103+2.1648901405887317e-17j), (0.17677669529663678-2.1648901405887307e-17j), (0.17677669529663675-2.164890140588731e-17j), (0.17677669529663678-2.1648901405887307e-17j), (-0.5303300858899103+2.1648901405887317e-17j)]



Create a magnitude amplification circuit for n = 3 qubits and single good outcome 5
using the circuit returned by the function prepare_binomial from chapter 4 as the
initial state-preparation operator (with theta = 4*pi/7):

In [135]:
theta = 4*pi/7

def prepare_binomial(n, theta):
  q = QuantumRegister(n)
  qc = QuantumCircuit(q)
  for i in range(len(q)):
    qc.ry(theta, q[i])
  return qc

In [136]:
n = 3 # num qubits
items = [5] # good outcomes

num_iterations = int(floor(pi/4*sqrt(2**n/len(items)))) # opt number of iterations

qc = grover_circuit(prepare_binomial(n, theta), phase_oracle_match(n, items), num_iterations)

In [137]:
# print out amplitudes
for name, (start, ops, end_state, _) in qc.reports.items():
    print(f"=== Report: {name} ===")
    print(" amplitudes:", end_state)
    print()

=== Report: iteration_1 ===
 amplitudes: [(-0.10155674247108569+1.2437113959904715e-17j), (-0.12734812708096677+1.5595647620711648e-17j), (-0.12734812708096693+1.5595647620711645e-17j), (-0.15968950043516228+1.955632355653612e-17j), (-0.12734812708096685+1.5595647620711645e-17j), (-0.9219188353157386+6.622940931542181e-17j), (-0.1596895004351622+1.9556323556536115e-17j), (-0.20024429988686446+2.4522854090395117e-17j)]

=== Report: iteration_2 ===
 amplitudes: [(-0.0982654627201712+2.4068096877398235e-17j), (-0.12322099281316773+3.0180438887280976e-17j), (-0.12322099281316769+3.018043888728095e-17j), (-0.15451423775513332+3.784507333790341e-17j), (-0.1232209928131677+3.018043888728096e-17j), (0.9270940979957676-1.6084315460836203e-16j), (-0.15451423775513334+3.784507333790343e-17j), (-0.19375472574912242+4.745622094166717e-17j)]

